In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
import pandas as pd

In [2]:
mag_dir = r'N:\2_Analysis\automated_analysis\Flint_SOT_EOT_volume\nii files\FLINT_EOT_NII_CORRECTED_AFFINE'
seg_dir = r'N:\2_Analysis\automated_analysis\Flint_SOT_EOT_volume\nii files\FLINT_EOT_WHOLE_LIVER_MASKS_NII\corrected'

mag_paths = []
seg_paths = []

for root, dirs, files in os.walk(mag_dir):
    for file in files:
        if file.endswith('.nii') and not file.startswith('f_5478'):
            mag_paths.append(file[:6])
            
for root, dirs, files in os.walk(seg_dir):
    for file in files:
        if file.endswith('.nii'):
            seg_paths.append(os.path.join(file[:6]))
            
mag_paths = sorted(mag_paths)
seg_paths = sorted(seg_paths)

print(len(mag_paths)), print(len(seg_paths))

missing = sorted(list(set(mag_paths) - set(seg_paths)))
print(missing)

81
81
[]


In [3]:
columns = {'Patient ID': [], 'liver_volume': [], 'N_voxels': [], 'pixel_spacing x': [], 'pixel_spacing y': [], 'slice_spacing': []}

for i in mag_paths:
    mag_ni = nib.load(os.path.join(mag_dir, i + '_EOT.nii'))
    seg = nib.load(os.path.join(seg_dir, i + '_EOT_whole_liver.nii')).get_fdata()
    
    voxel_dimensions = (mag_ni.affine[0,0],mag_ni.affine[1,1],mag_ni.affine[2,2])
    voxel_volume = voxel_dimensions[0] * voxel_dimensions[1] * voxel_dimensions[2]
    
    patient_id = i
    liv_vol = str(round((seg.sum() * voxel_volume) / 1000, 1))
    n_voxels = seg.sum()
    ps_x, ps_y, ps_z = mag_ni.affine[0,0], mag_ni.affine[1,1], mag_ni.affine[2,2]
    
    columns['Patient ID'].append(patient_id)
    columns['liver_volume'].append(liv_vol)
    columns['N_voxels'].append(n_voxels)
    columns['pixel_spacing x'].append(ps_x)
    columns['pixel_spacing y'].append(ps_y)
    columns['slice_spacing'].append(ps_z)

In [4]:
df = pd.DataFrame(columns)
df.shape

(81, 6)

In [5]:
df.to_excel(os.path.join(r'N:\2_Analysis\automated_analysis\Flint_SOT_EOT_volume',
                         'Flint_EOT_Volume.xlsx'), index = False, header = True)